# PocketXMol Small Molecule Design Interface

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pengxingang/PocketXMol/blob/master/notebooks/PXM_SmallMoleculeDesign.ipynb)

<!-- **Paper**: [PocketXMol](https://www.biorxiv.org) -->

**GitHub**: [PocketXMol](https://github.com/pengxingang/PocketXMol)

---

### This notebook provides an interface to run PocketXMol's **small molecule design** capability. You can:
- #### Design ***de novo*** small molecules binding to a protein pocket.
- #### Design pocket-binding molecules based on **fixed fragments**.
- #### Design pocket-binding molecules based on **unfixed fragments**.


> The notebook handles:
> 1. 🔧 Setting up the environment with required dependencies
> 2. ⚙️ Configuring for your specific task and protein receptor
> 3. 🔄 Running the sampling to generate small molecules
> 4. 📊 Visualize results

Let's get started!


---
## **1. Setup**

In [ ]:
#@title **Install Conda Colab**
#@markdown It will install Conda and restart the kernel. Don't worry.
!pip install -q condacolab
import condacolab
# condacolab.install()
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

In [ ]:
#@title **Setup PocketXMol**
#@markdown It will automatically load the PocketXMol model and setup the environment (takes several minutes).

import os
%cd /content
![ -d sample_data ] && rm -rf sample_data

# Clone the repository
if not os.path.exists('PocketXMol'):
    !echo Clone PocketXMol from github
    !git clone https://github.com/pengxingang/PocketXMol.git -q
else:
    print("✅ PocketXMol repository already exists")

# Download data
if not os.path.exists('/content/PocketXMol/data/trained_models/pxm/checkpoints/pocketxmol.ckpt'):
    %cd /content/PocketXMol
    # !gdown 1PF4V5kB-RLEFBD38HggVD9eR7RTeq573  # data_test.tar.gz
    !gdown 1Hu6qTkCyNUPPsQLLHL1kBFiwRbKUOFLs   # model_weights.tar.gz
    !tar -zxf model_weights.tar.gz && rm model_weights.tar.gz
else:
    print("✅ PocketXMol model weights already exists")
%cd /content

# Install dependencies
env_name = 'pxm_cu126'
install_path = f'install_{env_name}.sh'

cmd_list = [
    f"mamba create -n {env_name} python=3.11",
    f"source activate {env_name}",
    "pip install torch==2.6.0 --index-url https://download.pytorch.org/whl/cu126",
    "pip install biopython==1.83 rdkit==2023.9.3 peptidebuilder==1.1.0",
    "pip install lmdb easydict==1.9 numpy==1.24 pandas==1.5.2 scipy==1.10.1",
    "pip install pytorch-lightning==2.3",
    "pip install torch_geometric",
    "pip install torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.6.0+cu126.html",
    "pip install tensorboard",
    "mamba install -c conda-forge openbabel -y",
    "pip install 'setuptools<75.0'"
]
n_cmd = len(cmd_list)
with open(install_path, 'w+') as f:
    for i_cmd, cmd in enumerate(cmd_list):
        f.write(f"echo \"Running ({i_cmd+1}/{n_cmd})... >{cmd}\"\n")
        f.write(f"{cmd} >> install.log\n")

print("📦 Installing environment...")
!bash {install_path}
print("✅ PocketXMol environment has been setup")


## **2. Configure Small Molecule Design Task**

Configure your small molecule design task below, including:
1. **Job parameters**: set the job name and the sampling parameters.
2. **Input data**: provide input data including initial molecule and protein and define the pocket.
3. **Advanced settings (optional)**: configure the model for specific settings.


In [ ]:
# @title **Job parameters** {"run":"auto"}

#@markdown ### 1. Define the **job**
task_name = 'smallmol_design_example'  #@param {type:"string"}
output_directory = '/content/outputs'  #@param {type:"string"}

#@markdown ### 2. Configure **sampling** parameters
#@markdown Total number of small molecules to generate
num_samples = 50 #@param {type:"integer"}
#@markdown Number of poses to generate in each batch (reduce if you encounter memory issues)
batch_size = 50 #@param {type:"integer"}
#@markdown Ratio of molecules to save the sampling trajactory.
save_traj_prob = 0.05 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
#@markdown Random seed
seed = 2024 #@param {type:"integer"}
#@markdown Use GPU or not for sampling
device = "cuda:0" #@param ["cuda:0", "cpu"]

In [ ]:
# @title **Define input data** {"run":"auto"}

#@markdown ### 1. Provide the file path of **protein receptor**
protein_path = 'PocketXMol/data/examples/smallmol/2ar9_A.pdb'  #@param {type:"string"}
#@markdown > example: `PocketXMol/data/examples/smallmol/2ar9_A.pdb`

#@markdown ### 2. Provide the **ligand**
task_mode = 'de novo design'  #@param ["de novo design", "fixed-fragment based", "unfixed-fragment based"]
#@markdown Enter initial molecule file path or none for *de novo* design
input_ligand_path = "none"  #@param {type:"string"}
#@markdown > examples:
#@markdown > - `none` for *de novo* small molecule design
#@markdown > - `PocketXMol/data/examples/smallmol/fragment.sdf` for fragment-based design

#@markdown Specify the number of atoms of the generated molecules, as the mean and std of a Gaussian distribution:
num_atoms_mean = 28  #@param {type:"number"}
num_atoms_std = 2  #@param {type:"number"}
#@markdown > example: mean=`28`, std=`3`.

#@markdown For fragment-based design:
#@markdown - specifiy the reserved fragment atoms in the `input_ligand` (atom indices, counting from 0)
fragment_atom_indices = "0,1,2,3,4,5,6"  #@param {type:"string"}
#@markdown > example: `0,1,2,3,4,5,6`


#@markdown ### 3. Specify the **pocket**
#@markdown Two steps to define the pocket:

#@markdown **Step 1**: Specify the radius (Å) around the reference to define pocket (Å):
radius = 15 #@param {type:"slider", min:5.0, max:25.0, step:1.0}
#@markdown > examples: `15`, `13`

#@markdown **Step 2**: Choose either way to define a reference:
#@markdown 1. Provide *a reference molecule* around the pocket
#@markdown 2. Provide *a point coordinate* around the pocket as reference.
define_pocket_by = 'pocket_coordinate' #@param ["reference_molecule", "pocket_coordinate"]

#@markdown - If you choose `reference_molecule`, provide the reference file or directly use input ligand file as refernce
#@markdown (in this case, the input ligand (`input_ligand_path`) should be SDF file so that it contains correct coordinate information).
use_ligand_as_ref = False #@param {type:"boolean"}
reference_path = '' #@param {type:"string"}
#@markdown > example: `PocketXMol/data/examples/smallmol/fragment.sdf`

#@markdown - If you choose `pocket_coordinate`, provide the XYZ coordinate:
pocket_x = -8.1603 #@param {type:"number"}
pocket_y = 36.6972 #@param {type:"number"}
pocket_z = 38.7714 #@param {type:"number"}
#@markdown > example: `[-8.1603, 36.6972, 38.7714]`

#@markdown > 💡 Tips:
#@markdown > 1. Choose either value for `define_pocket_by` and set corresponding parameters. Make sure their consistency.
#@markdown > 2. The radius of $[10, 16]$ is recomendded for `define_pocket_by=reference_molecule`
#@markdown > 3. The radius of $[12, 20]$ is recomendded for `define_pocket_by=pocket_coordinate`. In this case, the radius can be slightly larger to cover engough pocket residues.


# Advanced settings, default values (in case people forget to run the next cell)
space_center_by = 'pocket_center'
num_steps = 100
refine_based = True


### **Advanced settings (optional)**

In [ ]:
# @title  {"run":"auto"}


#@markdown ### Refinement of de novo design
#@markdown The parameter controls whether use the refinement rounds for designing.
#@markdown The default value is True, which is recommended for improved quality at the cost of speed.
refine_based = True #@param {type:"boolean"}

# #@markdown ### Initial noise scale
# #@markdown - The parameter controls the initial noise scale (in $[0, 1]$), from which the noise scale gradually decays to zero during sampling.
# #@markdown  - $\text{init_noise_scale}=0$ means no noise at the initial step (of course, also no noise at subsequent steps)
# #@markdown  - $\text{init_noise_scale}=1$ means sampling from noise prior at the initial step (input ligand information (if any) is ignored).
# #@markdown  - $0<\text{init_noise_scale}<1$ means adding noise to the input ligand at the initial step (instead of from noise prior).
# #@markdown - For *de novo* design, set $\text{init_noise_scale}=1$; if you want to use the information of the input ligand (e.g., generating similar ligand), set $\text{init_noise_scale}<1$.
# init_noise_scale = 1 #@param {type:"number"}

#@markdown ### Noise space center
#@markdown - The parameter defines the noise space center during sampling
#@markdown  - `pocket_center` directly uses the pocket center as space center
#@markdown  - `input_ligand_center` uses the input ligand center as space center. In this case, the input ligand should be SDF file that contains correct coordinate information.
#@markdown  - `specified_coordinate` uses the provided coordinate as space center
#@markdown - Generally, this parameter has little impact so it is recommended to use `pocket_center` (default) or `input_ligand_center`.
space_center_by = 'pocket_center' #@param ["pocket_center", "input_ligand_center", "specified_coordinate"]
#@markdown  - Specify the coordinate if you choose `specified_coordinate`, otherwise, skip it.
space_x = 0.0 #@param {type:"number"}
space_y = 0.0 #@param {type:"number"}
space_z = 0.0 #@param {type:"number"}


#@markdown ### Denoising steps
#@markdown - The parameter defines the denosing steps for sampling
#@markdown - Default is 100.
num_steps = 100 #@param {type:"integer"}



## **3. Run sampling**

Now run the sampling program with your configuration.

In [ ]:
#@title **Prepare config file**
#@markdown The previous parameters will be saved in the config file.
import os
import yaml


print('Pareparing config file...')

# config file path
protein_path = os.path.abspath(protein_path)
if os.path.isfile(input_ligand_path):
  input_ligand = os.path.abspath(input_ligand_path)
elif input_ligand_path in ['none', 'None', 'NONE']:
  input_ligand = None
else:
  raise ValueError(f'Invalid input_ligand value:', input_ligand_path)

# setup config/data/pocket_args
pocket_args = {"radius": radius}
if define_pocket_by == 'reference_molecule':
  if use_ligand_as_ref:
    assert input_ligand.endswith('.sdf'), 'If the input ligand is used as pocket reference, it must be PDB file that contains coordinate information.'
    pocket_args['ref_ligand_path'] = input_ligand
    print('Use input_ligand as pocket reference')
  else:
    reference_path = os.path.abspath(reference_path)
    pocket_args['ref_ligand_path'] = reference_path
    print(f'Use provided reference molecule in {reference_path} as pocket reference')
elif define_pocket_by == 'pocket_coordinate':
  pocket_coord = [pocket_x, pocket_y, pocket_z]
  pocket_args['pocket_coord'] = pocket_coord
  print(f'Use pocket coordinate {pocket_coord} as pocket reference')
else:
  raise ValueError(f'Invalid value for define_pocket_by: {define_pocket_by}.')

# task_mode
config_dir = '/content/PocketXMol/configs/sample/examples'
if task_mode == 'de novo design':
  if refine_based:
    config_path = os.path.join(config_dir, 'sbdd.yml')
  else:
    config_path = os.path.join(config_dir, 'sbdd_simple.yml')
elif task_mode == 'fixed-fragment based':
  config_path = os.path.join(config_dir, 'growing_fixed_frag.yml')
elif task_mode == 'unfixed-fragment based':
  config_path = os.path.join(config_dir, 'growing_unfixed_frag.yml')
else:
  raise ValueError(f'Invalid value {task_mode} for task_mode')
with open(config_path, 'r') as f:
  config = yaml.safe_load(f)


# setup config/transforms
transforms = {
    "variable_mol_size":{  # distributions of number of atoms
      "name": "variable_mol_size",
      "num_atoms_distri": {
        "strategy": "mol_atoms_based",
        "mean": {
          "coef": 0,
          "bias": num_atoms_mean,
        },
        "std": {
          "coef": 0,
          "bias": num_atoms_std,
        }
      }
    }
}
if space_center_by == 'pocket_center':
  pass
elif space_center_by == 'input_ligand_center':
  transforms['featurizer'] = {'mol_as_pocket_center': True}
elif space_center_by == 'specified_coordinate':
  transforms['featurizer_pocket'] = {'center': [space_x, space_y, space_z]}
else:
  raise ValueError(f'Invalid space_center_by: {space_center_by}.')

data_id = '_'.join(task_name.split())
config["sample"] = {
    "seed": seed,
    "batch_size": batch_size,
    "num_mols": num_samples,
    "save_traj_prob": save_traj_prob,
}
config["data"] = {
    "protein_path": protein_path,
    "input_ligand": input_ligand,
    "is_pep": False,
    "pocket_args": pocket_args,
    "pocmol_args": {
        "data_id": data_id,
        "pdbid": ""
    }
}
config["noise"]["num_steps"] = num_steps


# fragment
if task_mode == 'fixed-fragment based':
  fragment_atom_indices = [int(ia) for ia in fragment_atom_indices.split(',')]
  config['task']['transform']['preset_partition']['grouped_node_p1'] = [fragment_atom_indices]
elif task_mode == 'unfixed-fragment based':
  fragment_atom_indices = [int(ia) for ia in fragment_atom_indices.split(',')]
  config['task']['transform']['preset_partition']['init_frag'] = fragment_atom_indices


# Save configuration to file
config_dir = f"/content/PocketXMol/configs/user_defined"
!mkdir -p {config_dir}
config_path = f"{config_dir}/{data_id}.yml"
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, indent=2)
print(f"✅ Configuration file created at {config_path}\n")


print("Configuration Summary for Small Molecules Design:")
print(f"- Protein File: {protein_path}")
print(f"- Ligand: {input_ligand}")
print(f"- Task Mode: {task_mode}")
print(f"- Pocket Radius: {radius} Å")
print(f"- Generate {num_samples} small molecules with batch size of {batch_size}")

In [ ]:
#@title **Generate small molecules poses**
#@markdown Running the PocketXMol sampling script


output_dir = output_directory

# # Create output directory
!mkdir -p {output_dir}

print(f"🚀 Starting sampling...")
print(f"This may take some time depending on your configuration.")

# Run
!cd /content/PocketXMol && source activate {env_name} && \
python scripts/sample_use.py \
    --config_task {config_path} \
    --outdir {output_dir} \
    --device {device}


# get exp_dir
from datetime import datetime
import re
exp_name_list = [f for f in os.listdir(output_dir) if f.startswith(data_id)]
def extract_timestamp(filename):
    match = re.search(r'(\d{8}_\d{6})$', filename)
    if match:
        return datetime.strptime(match.group(1), "%Y%m%d_%H%M%S")
    return datetime.min
exp_name = max(exp_name_list, key=lambda f: extract_timestamp(f))
gen_path = f'{output_dir}/{exp_name}'

print(f"✅ Sampling completed! Results saved to {gen_path}")



📋 Now there are the following content in the generation directory:
- `{exp_name}_SDF`: the SDF/PDB files of the generated ligand poses
- `SDF`: the generation trajactory files (Non-empty only if `save_traj_prob>0`)
- `gen_info.csv`: the meta information of the generated poses, including file names, self-confidence score (`cfd_traj`), and others.
- `log.txt`: generation logs
<!-- - `scripts`,`utils`,`models`: copy of core codes when running. -->
- `{job_name}.yml`: the complete config file.

## **4. Analyze Results**



In [ ]:
#@title **Show top-ranked results**
import pandas as pd
import numpy as np
import shutil
from scipy.special import expit  # sigmoid


# Find ranking file if it exists (for small molecules)
gen_info = os.path.join(gen_path, "gen_info.csv")
gen_lig_dir = os.path.join(gen_path, f'{os.path.basename(gen_path)}_SDF')
pocket_path = os.path.join(gen_lig_dir, '0_inputs/pocket_block.pdb')
df_gen = pd.read_csv(gen_info)

# Sort by tuned_ranking score (higher is better)
df_gen = df_gen.sort_values(by="cfd_traj", ascending=False)
df_gen = pd.concat([df_gen[df_gen['tag'].isna()],
                    df_gen[df_gen['tag'].notna()]], ignore_index=True)
df_gen['cfd_traj_prob'] = df_gen['cfd_traj'].apply(expit)


# Show top values
n_top = 10
lines_top = df_gen.head(n_top)
lines_top.insert(0, 'rank', np.arange(n_top))
# make top subdirectory
top_dir = os.path.join(gen_path, 'top_ranks')
os.makedirs(top_dir, exist_ok=True)
for i_sort, (_, line) in enumerate(lines_top.iterrows()):
  filename = line['filename']
  src_path = os.path.join(gen_lig_dir, filename)
  tgt_path = os.path.join(top_dir, f'rank{i_sort}_{filename}')
  shutil.copy(src_path, tgt_path)
lines_top.to_csv(os.path.join(top_dir, f'gen_info_top{n_top}.csv'), index=False)
shutil.copy(pocket_path, os.path.join(top_dir, 'pocket_block.pdb'))

print(f"Top {n_top} poses:")
lines_top

In [ ]:
# @title **Show top-ranked 3D structure** {"run":"auto"}
receptor_style = "surface" #@param ["line", "surface", "cartoon", "stick"]
show_receptor = "pocket" #@param ["protein", "pocket", "none"]
ligand_style = "stick"
show_ligand_rank = 0 # @param {type:"slider", min:0, max:9, step:1}


# Install required packages
try:
  import py3Dmol
except ModuleNotFoundError:
  os.system('pip install py3dmol')
  import py3Dmol
try:
  from rdkit import Chem
  from rdkit.Chem import AllChem
except ModuleNotFoundError:
  os.system('pip install rdkit')
  from rdkit import Chem
  from rdkit.Chem import AllChem

import ipywidgets as widgets
from IPython.display import display


# Prepare file
lig_filename_list = df_gen['filename'].values.tolist()


# Step : Visualization function
def show_complex(receptor_paths, ligand_paths):
    viewer = py3Dmol.view(width=800, height=600)

    # Load receptors
    for receptor_path in receptor_paths:
      with open(receptor_path, 'r') as f:
          viewer.addModel(f.read(), 'pdb')
      # Apply style options
      style = {}
      if receptor_style == 'cartoon':
          style = {"cartoon": {"color": "spectrum"}}
          viewer.setStyle(style)
      elif receptor_style == 'stick':
          style = {"stick": {"colorscheme": "greenCarbon"}}
          viewer.setStyle(style)
      elif receptor_style == 'surface':
          viewer.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'color': 'white'}, {'model': 0})

    # Load ligands
    for ligand_path in ligand_paths:
      if ligand_path.endswith('.sdf'):
        if ligand_style == 'cartoon': continue
        suppl = Chem.SDMolSupplier(ligand_path)
        for mol in suppl:
          if mol is None:
            continue
          mol_block = Chem.MolToMolBlock(mol)
          viewer.addModel(mol_block, 'mol')
          if ligand_style == 'surface':
            viewer.addSurface(py3Dmol.VDW, {'opacity': 0.9, 'color': "spectrum"}, {'model': -1})
          else:
            viewer.setStyle({'model': -1}, {"stick": {}})
      else:  # assume PDB
        if ligand_style == 'stick': continue
        with open(ligand_path, 'r') as f:
          viewer.addModel(f.read(), 'pdb')
          viewer.setStyle({'model': -1}, {"cartoon": {"color": "spectrum"}})


    # set viewer
    viewer.zoomTo({'model': -1})
    return viewer


if show_ligand_rank < num_samples:
  show_lig_names = [lig_filename_list[show_ligand_rank]]
  show_lig_paths = [os.path.join(gen_lig_dir, name) for name in show_lig_names]
  if show_lig_paths[0].endswith('.pdb'):
    show_lig_paths.append(show_lig_paths[0].replace('.pdb', '_mol.sdf'))
  print('Showing ligand pose path:', show_lig_paths[0])
else:
  raise ValueError(f'`show_ligand_rank`={show_ligand_rank} cannot exceed `num_samples`={num_samples}.')
show_rec_paths = []
if show_receptor == 'protein':
  show_rec_paths.append(protein_path)
  print('Showing Receptor (protein):', protein_path)
elif show_receptor ==  'pocket':
  show_rec_paths.append(pocket_path)
  print('Showing Receptor (pocket):', pocket_path)

show_complex(show_rec_paths, show_lig_paths).show()

In [ ]:
#@title **Download Results**
#@markdown Download the generation results.

from google.colab import files
import os

zip_filename = f"PXM_{exp_name}.zip"

# Zip the results directory
!cd {output_dir} && zip -r {zip_filename} {exp_name} -q
# Download the zip file
files.download(os.path.join(output_dir, zip_filename))
print(f"Downloaded {zip_filename} containing generation results")

